In [1]:
import pandas as pd

In [2]:
# Gold
df = pd.read_csv("Gold.csv",sep = ",")
df['Date Local'] = pd.to_datetime(df['Date Local'], errors='coerce')
df_jan = df[(df['Date Local'] >= '2019-01-01') & (df['Date Local'] <= '2019-01-31')]
df_jan = df_jan[['Latitude', 'Longitude', 'AQI']]
df_avg = df_jan.groupby(['Latitude', 'Longitude'], as_index=False).mean()
df_avg.to_csv("Gold_clean.csv", index=False)

/var/folders/9n/7bxqt9q92tl6lchpfwnymvjh0000gn/T/ipykernel_5704/256820174.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Gold.csv",sep = ",")


In [3]:
#CMAQ
df3 = pd.read_csv("CMAQ.csv", skiprows=2)
df3 = df3.iloc[1:]
df4 = df3[['latitude', 'longitude', 'PM25_AVG']].copy()
df4['latitude'] = pd.to_numeric(df4['latitude'], errors='coerce').round(6)
df4['longitude'] = pd.to_numeric(df4['longitude'], errors='coerce').round(6)
df4['PM25_AVG'] = pd.to_numeric(df4['PM25_AVG'], errors='coerce')
df_avg2 = df4.groupby(['latitude', 'longitude'], as_index=False).mean()
df_avg2.to_csv("CMAQ_clean.csv", index=False)

/var/folders/9n/7bxqt9q92tl6lchpfwnymvjh0000gn/T/ipykernel_5704/3912202928.py:2: DtypeWarning: Columns (2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("CMAQ.csv", skiprows=2)


In [4]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Get CVS1: 
# This is U2 Part (about 1000 data) si, Z1 and closest uj and Z2
df_gold = pd.read_csv("Gold_clean.csv")   # Gold: Latitude, Longitude, AQI
df_cmaq = pd.read_csv("CMAQ_clean.csv")   # CMAQ: latitude, longitude, PM25_AVG

df_gold = df_gold.rename(columns={'Latitude': 'si_lat', 'Longitude': 'si_lon', 'AQI': 'Z1'})
df_cmaq = df_cmaq.rename(columns={'latitude': 'uj_lat', 'longitude': 'uj_lon', 'PM25_AVG': 'Z2'})

X_gold = df_gold[['si_lat', 'si_lon']].to_numpy()
X_cmaq = df_cmaq[['uj_lat', 'uj_lon']].to_numpy()

nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nbrs.fit(X_cmaq)

distances, indices = nbrs.kneighbors(X_gold)

nearest_cmaq = df_cmaq.iloc[indices.flatten()].reset_index(drop=True)

df_gold_reset = df_gold.reset_index(drop=True)
df_matched = pd.concat([df_gold_reset, nearest_cmaq], axis=1)

df_csv1 = df_matched[['si_lat', 'si_lon', 'uj_lat', 'uj_lon', 'Z1', 'Z2']]

df_csv1.to_csv("CSV1.csv", index=False)

In [5]:
# U1/U2: uj, Z2 that are not in U2

df_cmaq = pd.read_csv("CMAQ_clean.csv")
df_cmaq = df_cmaq.rename(columns={'latitude': 'uj_lat', 'longitude': 'uj_lon', 'PM25_AVG': 'Z2'})

df_csv1 = pd.read_csv("CSV1.csv")  

# get ujs that are not in U2
df_cmaq_unique = df_cmaq.drop_duplicates(subset=['uj_lat', 'uj_lon']) 

df_unmatched = df_cmaq_unique.merge(
    df_csv1[['uj_lat', 'uj_lon']],
    on=['uj_lat', 'uj_lon'],
    how='left',
    indicator=True
)

df_unmatched = df_unmatched[df_unmatched['_merge'] == 'left_only'].drop(columns=['_merge'])

df_unmatched.to_csv("CSV2.csv", index=False)

In [6]:
from sklearn.model_selection import train_test_split
df_csv1_train, df_csv1_test = train_test_split(df_csv1, test_size=0.1, random_state=42)
df_csv1_train.to_csv("CSV1_train.csv", index=False)
df_csv1_test.to_csv("CSV1_test.csv", index=False)